# Preambolo
prova a considerare class weight balanced.
Fai albero ottimo 3 vs altri, 4 vs altri e fai il pdf-->10 iterazioni, ricordati di fare holdout
Per classe 4  fai split 70-30.
Tumori maligni --> 1 carcinoma, 2 linfoma , 5 è cisto adenoma ma non se se de preciso così e se è benigno o maligno.
Per ora 5 lo considero poco, è una classe un po' indeterminata

1,2 vs 3,4,5--> maligni contro benigni

f1-score--> meglio dell'accuratezza per classi non bilanciate
tu cerca di aggirare lo sbolanciamento tra classi--> nell'allenamento degli alberi considero lo weight pesando più le classi meno rappresentate-->peso inversamente proporzionale al numero

nella media dell'accuratezza visualizza anche dev std


## Moduli importati

In [1]:
import graphviz
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTENC, SMOTE
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer, precision_score

## Importo dataset

In [2]:
df = pd.read_excel('../Db_parotide_Def_REV_Samuel.xlsx')
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)
df.iat[0, 6] = 3
df.iat[1, 6] = 3
df.iat[20, 6] = 3
df.head()

,ID,eta,Segni macro malignità,ADC,TIC type,T2,Output Algoritmo,COD ISTO,ISTO DEFINITIVO-PULITO,ISTO DEFINITIVO,...,CONDORD CODICE FNAC-ISTO DIAGN,CONCORD multiRMN-ISTO B/M,CONCORD multiRMN-ISTO DIAGN,CONCORD FNAC-RMN,FNAC,DATA FNAC,ESITO FNAC,COD FNAC,MILAN,DATA INTERVENTO
0,4,33,0,2.60,D,0,3,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,27/12/2017,Positivo per CTM. Carcinoma squamocellulare o ...,3,6,24/01/2018
1,10,50,0,2.44,D,0,3,1,CARCINOMA,CARCINOMA ADENOIDECISTICO G3,...,1,1,1,1,0,27/12/2016,carcinoma adenoidecistico,3,6,27/11/2019
2,9,81,1,0.70,C,1,1,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,21/03/2019,"Positivo per CTM, carcinoma squamocellulare",3,6,15/05/2019
3,1,28,0,1.60,A,1,3,1,CARCINOMA,CARCINOMA DEI DOTTI SALIVARI,...,1,1,1,1,0,2016-06-10 00:00:00,sospetto per CTM,5,5,25/01/2017
4,7,51,0,1.40,A,0,3,1,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,...,1,1,1,1,0,13/09/2018,Positivo per CTM. Carcinoma squamocellulare,3,6,2018-10-10 00:00:00


## Samples e labels

In [3]:
X_not_encoded = df.loc[:,'eta':'T2'].copy()
X = pd.get_dummies(X_not_encoded, columns = ['TIC type'])
y = df['COD ISTO'].copy()
y.value_counts()

3    54
4    27
1    16
2     5
5     5
Name: COD ISTO, dtype: int64

## Funzione

In [4]:
#qui non divido in test set e training set ma faccio direttamente gridsearch
#non prevedo augmentation con smote

def GS(X, y, target_one, tuned_parameters, 
                scoring_func = accuracy_score, n_splits= 5,
                random_state_tree = None,
                n_it = 10, filename = 'albero'):
    
    scores = []
    models = []
    if scoring_func is f1_score:
        scoring = make_scorer(scoring_func, average = 'weighted')
    else:
        scoring = make_scorer(scoring_func)
        
    cv = StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = None)
    
    y = [1 if x in target_one else 0 for x in y]
    
    for n in range(n_it):
        
        print('### ITERATION %d ###\n\n' %(n+1))
        print('# Tuning hyper-parameters #\n')

        
        albero = tree.DecisionTreeClassifier(random_state = random_state_tree)
        

        clf = GridSearchCV(albero, tuned_parameters, scoring = scoring, cv = cv, refit = True)
        clf.fit(X, y)
        models.append(clf.best_estimator_)

        params = clf.best_params_
        score = clf.best_score_
 
 
        print("Parameters set found:\n")
        print(params)
        print()
        print("Mean cross-validated score of the estimator: %0.3f\n" %(score))
        print('-'*100)
        
        scores.append(score)
        
    print('\n\n# Final Mean score: %0.3f\n# Std dev: %0.3f' %(np.mean(scores), np.std(scores)))
    
    best_model = models[np.argmax(scores)]
    
    data = tree.export_graphviz(best_model, out_file=None, 
                      feature_names=X.columns,  
                      class_names=['0','1'],  
                      filled=True, rounded=True,  
                      special_characters=True) 
    graph = graphviz.Source(data) 
    graph.render('GS2-'+filename)
    

In [5]:
seed = 42 
tuned_parameters = {'criterion': ['gini', 'entropy'],
                    'max_depth': list(range(2,11)),     
                    'min_samples_leaf': list(range(2,11)),
                    'class_weight': ['balanced', None]}

In [6]:
GS(X,y, [3], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score, filename = '3-acc')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 10}

Mean cross-validated score of the estimator: 0.607

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 10}

Mean cross-validated score of the estimator: 0.589

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.559

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Pa

In [7]:
GS(X,y, [3], tuned_parameters, random_state_tree = seed, scoring_func = f1_score, filename = '3-f1')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 10}

Mean cross-validated score of the estimator: 0.516

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.586

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 6}

Mean cross-validated score of the estimator: 0.551

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters 

In [8]:
GS(X,y, [4], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score, filename = '4-acc')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.710

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.730

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.775

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Parameters

In [9]:
GS(X,y, [4], tuned_parameters, random_state_tree = seed, scoring_func = f1_score, filename = '4-f1')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.695

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.649

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.685

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Parameters se

In [10]:
GS(X,y, [1,2], tuned_parameters, random_state_tree = seed, scoring_func = accuracy_score, filename = '12-acc')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.804

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 5}

Mean cross-validated score of the estimator: 0.804

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.775

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Parameters se

In [11]:
GS(X,y, [1,2], tuned_parameters, random_state_tree = seed, scoring_func = f1_score, filename = '12-f1')

### ITERATION 1 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 4}

Mean cross-validated score of the estimator: 0.692

----------------------------------------------------------------------------------------------------
### ITERATION 2 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'entropy', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.717

----------------------------------------------------------------------------------------------------
### ITERATION 3 ###


# Tuning hyper-parameters #

Parameters set found:

{'class_weight': None, 'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2}

Mean cross-validated score of the estimator: 0.707

----------------------------------------------------------------------------------------------------
### ITERATION 4 ###


# Tuning hyper-parameters #

Parameters